In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("spark2")
  .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://10.110.48.55:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1680803218993)
SparkSession available as 'spark'


23/04/06 13:47:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5f30e7a1


In [2]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@46e24902


In [3]:
val test = spark.read.option("header", "true").csv("test.csv")
val train = spark.read.option("header", "true").csv("train.csv")

test: org.apache.spark.sql.DataFrame = [PassengerId: string, Pclass: string ... 9 more fields]
train: org.apache.spark.sql.DataFrame = [PassengerId: string, Survived: string ... 10 more fields]


### EDA

In [5]:
train.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [6]:
test.show(5)

+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|  47|    1|    0| 363272|      7| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|  62|    0|    0| 240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|  27|    0|    0| 315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|  22|    1|    1|3101298|12.2875| null|       S|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
only showing top 5 rows



In [9]:
import org.apache.spark.sql.functions.count

train.groupBy("Sex", "Survived")
     .agg(count("Survived").alias("count"))
     .show()


+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|female|       0|   81|
|female|       1|  233|
|  male|       1|  109|
|  male|       0|  468|
+------+--------+-----+



import org.apache.spark.sql.functions.count


In [12]:


train.groupBy("Pclass", "Survived")
     .agg(count("Survived").alias("count"))
     .show()

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|     1|       1|  136|
|     2|       0|   97|
|     3|       1|  119|
|     3|       0|  372|
|     2|       1|   87|
|     1|       0|   80|
+------+--------+-----+



In [16]:

train.groupBy("Pclass", "Sex")
     .agg(count("Sex").alias("count"))
     .show()

+------+------+-----+
|Pclass|   Sex|count|
+------+------+-----+
|     2|female|   76|
|     2|  male|  108|
|     3|  male|  347|
|     1|  male|  122|
|     1|female|   94|
|     3|female|  144|
+------+------+-----+



### Feature Engineering

In [17]:
val test_new = test
  .withColumn("Age", col("Age").cast("Double"))
  .withColumn("Fare", col("Fare").cast("Double"))
  .withColumn("Pclass", col("Pclass").cast("Integer"))
  .withColumn("PassengerId", col("PassengerId").cast("Integer"))

test_new: org.apache.spark.sql.DataFrame = [PassengerId: int, Pclass: int ... 9 more fields]


In [68]:
val train_new = train
  .withColumn("Age", col("Age").cast("Double"))
  .withColumn("Fare", col("Fare").cast("Double"))
  .withColumn("Pclass", col("Pclass").cast("Integer"))
  .withColumn("Survived", col("Survived").cast("Integer"))
  .withColumn("PassengerId", col("PassengerId").cast("Integer"))

train_new: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 10 more fields]


In [71]:
val ageaggtrain = train_new.agg(avg("Age")).first()(0).asInstanceOf[Double]
val fareaggtrain = train_new.agg(avg("Fare")).first()(0).asInstanceOf[Double]
val train_final = train_new.na.fill(ageaggtrain, Seq("Age")).na.fill(fareaggtrain, Seq("Fare"))

ageaggtrain: Double = 29.69911764705882
fareaggtrain: Double = 32.2042079685746
train_final: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 10 more fields]


In [72]:
val ageagg = test_new.agg(avg("Age")).first()(0).asInstanceOf[Double]
val fareagg = test_new.agg(avg("Fare")).first()(0).asInstanceOf[Double]
val test_final = test_new.na.fill(ageagg, Seq("Age")).na.fill(fareagg, Seq("Fare"))

ageagg: Double = 30.272590361445783
fareagg: Double = 35.6271884892086
test_final: org.apache.spark.sql.DataFrame = [PassengerId: int, Pclass: int ... 9 more fields]


In [18]:
test_new.show(5)

+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875| null|       S|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
only showing top 5 rows



### Prediction

In [21]:
val women = train.filter(col("Sex") === "female").select("Survived")
val rate_women = women.agg(avg("Survived")).as[Double].head()
println("% of women survived: " + rate_women)


% of women survived: 0.7420382165605095


women: org.apache.spark.sql.DataFrame = [Survived: string]
rate_women: Double = 0.7420382165605095


In [23]:
val women = train.filter(col("Sex") === "male").select("Survived")
val rate_men = women.agg(avg("Survived")).as[Double].head()
println("% of men survived: " + rate_men)


% of men survived: 0.18890814558058924


women: org.apache.spark.sql.DataFrame = [Survived: string]
rate_men: Double = 0.18890814558058924


In [78]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder, VectorAssembler}
import org.apache.spark.ml.classification.LogisticRegression

val sexIndexer = new StringIndexer()
  .setInputCol("Sex")
  .setOutputCol("SexIndex")

val embarkedIndexer = new StringIndexer()
  .setInputCol("Embarked")
  .setOutputCol("EmbarkedIndex")

val sexEncoder = new OneHotEncoder()
  .setInputCol("SexIndex")
  .setOutputCol("SexVec")

val embarkedEncoder = new OneHotEncoder()
  .setInputCol("EmbarkedIndex")
  .setOutputCol("EmbarkedVec")

val assembler = new VectorAssembler()
  .setInputCols(Array("Pclass", "Age", "Fare"))
  .setOutputCol("features")

val lr = new LogisticRegression()
  .setLabelCol("Survived")
  .setFeaturesCol("features")
  .setMaxIter(10)

val pipeline = new Pipeline()
  .setStages(Array(sexIndexer, embarkedIndexer, sexEncoder, embarkedEncoder, assembler, lr))

val pipelineModel = pipeline.fit(train_final)

val predictions = pipelineModel.transform(test_final)


23/04/06 14:48:22 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder, VectorAssembler}
import org.apache.spark.ml.classification.LogisticRegression
sexIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_11e1809eeda4
embarkedIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_17316ab1fba1
sexEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_34b6c30464cd
embarkedEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_38baf25c479b
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_8e210e4ce38e, handleInvalid=error, numInputCols=3
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_a35e622db1da
pipeline: org.apache.spark.ml.Pipeline = pipeline_7c06efe6a676
pipel...


In [80]:
predictions.show()

+-----------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+--------+-------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|PassengerId|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|SexIndex|EmbarkedIndex|       SexVec|  EmbarkedVec|            features|       rawPrediction|         probability|prediction|
+-----------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+--------+-------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|        892|     3|    Kelly, Mr. James|  male|              34.5|    0|    0|          330911| 7.8292| null|       Q|     0.0|          2.0|(1,[0],[1.0])|    (2,[],[])|   [3.0,34.5,7.8292]|[1.45294447734381...|[0.81045117832216...|       0.0|
|        893|     3|